In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, GenerationConfig
from Fast_dLLM_v2_7B.modeling import Fast_dLLM_QwenForCausalLM

model_name = "Efficient-Large-Model/Fast_dLLM_7B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# remote config (no remote code execution)
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)

# using local class to load remote weights
model = Fast_dLLM_QwenForCausalLM.from_pretrained(
    model_name, 
    config=config, 
    trust_remote_code=True,
    dtype="auto",
    device_map="auto",)  # downloads weights from Hub

# (optional) generation parameters from the repo
gen_config = GenerationConfig.from_pretrained(model_name)
model.generation_config = gen_config


2025-11-28 20:42:32.963723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 20:42:33.026560: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 20:42:34.756455: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

gen_text = "A large language model is"

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

text = text + gen_text
inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Fast-dLLM v2 parallel decoding
gen_ids, past_key_values, past_block_key_values = model.generate(
    inputs["input_ids"],
    tokenizer=tokenizer,
    max_new_tokens=256,
    small_block_size=8,
    threshold=0.95,
    steps=18,
)

response = tokenizer.decode(
    gen_ids[0][inputs["input_ids"].shape[1]:], 
    skip_special_tokens=False
)
print(response)

 a type of artificial intelligence model designed to process and generate human language.|<MASK>| are typically trained on massive|<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK

In [4]:
print(tokenizer.decode(inputs["input_ids"][0][:-1], skip_special_tokens=False))
print(inputs["input_ids"])
print(gen_ids)



<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
A large language model
tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
          16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
          77091,    198,     32,   3460,   4128,   1614,    374]])
tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
          16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
          77091,    198,     32,   3460,   4128,   1614,    374,    264,    943,
            315,  20443,  11229,   1614,   6188,    311,   1882,    323,   6923,
           3738,   4128,     13, 151665,    525,  11136,  16176,    389,  10951,
         151665, 151

In [5]:

import torch

mask_id = 151665
indices = torch.where(gen_ids[0] == mask_id)[0]

stripped_ids = gen_ids[0][:indices[0]].unsqueeze(0)

print(stripped_ids)

gen_ids, past_key_values, past_block_key_values = model.generate(
    stripped_ids,
    tokenizer=tokenizer,
    max_new_tokens=236,
    small_block_size=8,
    threshold=0.95,
    steps=18,
)

response = tokenizer.decode(
    gen_ids[0], 
    skip_special_tokens=False
)
print(response)



tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
          16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
          77091,    198,     32,   3460,   4128,   1614,    374,    264,    943,
            315,  20443,  11229,   1614,   6188,    311,   1882,    323,   6923,
           3738,   4128,     13]])


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
A large language model is a type of artificial intelligence model designed to process and generate human language. They are typically trained on massive datasets of text, allowing them to learn intricate patterns,|<MASK>|,|<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||<MASK>||

In [6]:
print(past_key_values)

DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer])


In [9]:
print(past_key_values.layers[0].values)

tensor([[[[ 0.1367, -0.0767, -0.2402,  ...,  0.2637,  0.1406,  0.0684],
          [ 0.1592,  0.1182, -0.6016,  ...,  0.1128,  0.0608,  0.2617],
          [-0.0250,  0.1152, -0.0649,  ...,  0.0173,  0.0334, -0.0238],
          ...,
          [ 0.2949, -0.0040,  0.4004,  ...,  0.3223,  0.0403, -0.1074],
          [-0.0086,  0.1011, -0.0386,  ...,  0.1973,  0.0403, -0.3711],
          [ 0.1709, -0.1074,  0.1084,  ..., -0.2988,  0.4375, -0.0654]],

         [[-0.0723,  0.0366,  0.7109,  ...,  0.1533,  0.0664,  0.0942],
          [-0.0576,  0.0352,  1.0391,  ...,  0.4570, -0.3672,  0.3691],
          [ 0.1426, -0.1553, -0.3711,  ...,  0.1025,  0.0209,  0.0283],
          ...,
          [ 0.5547, -0.0376,  0.7305,  ..., -0.6602,  0.0277, -0.0500],
          [-0.5586,  0.0515,  1.4297,  ...,  0.0503,  0.3047, -0.4199],
          [ 0.0664,  0.1855,  0.5938,  ...,  0.4648,  0.0035, -0.7305]],

         [[-0.1147, -0.0498, -0.1758,  ...,  0.1260, -0.2305, -0.3535],
          [ 0.1602,  0.1670,  

In [14]:
from transformers import DynamicLayer, DynamicCache

def truncate_dynamic_cache(dynamic_cache, new_len):
    """
    Truncates a HuggingFace DynamicCache to match a shortened sequence length.
    """
    for i in range(len(dynamic_cache)):
        if dynamic_cache.layers[i].keys is not None:
            dynamic_cache.layers[i].keys = dynamic_cache.layers[i].keys[:, :, :new_len, :].contiguous()
        if dynamic_cache.layers[i].values is not None:
            dynamic_cache.layers[i].values = dynamic_cache.layers[i].values[:, :, :new_len, :].contiguous()
    return dynamic_cache

def truncate_block_cache(block_cache, new_len, block_size):
    max_blocks = (new_len + block_size - 1) // block_size
    return block_cache[:max_blocks]

In [15]:
new_seq_len = stripped_ids[0].size(0)
print(new_seq_len)

past_key_values_trunc = truncate_dynamic_cache(past_key_values, new_seq_len)
print(past_key_values_trunc)

48
DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer])


In [18]:
mask_id = 151665
indices = torch.where(gen_ids[0] == mask_id)[0]

stripped_ids = gen_ids[0][:indices[0]].unsqueeze(0)

print(stripped_ids)

gen_ids, past_key_values, past_block_key_values = model.generate(
    stripped_ids,
    tokenizer=tokenizer,
    max_new_tokens=236,
    small_block_size=8,
    threshold=0.95,
    steps=18,
    past_key_values=past_key_values_trunc
)

response = tokenizer.decode(
    gen_ids[0], 
    skip_special_tokens=False
)
print(response)

tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198,  35127,    752,    264,   2805,
          16800,    311,   3460,   4128,   1614,     13, 151645,    198, 151644,
          77091,    198,     32,   3460,   4128,   1614,    374,    264,    943,
            315,  20443,  11229,   1614,   6188,    311,   1882,    323,   6923,
           3738,   4128,     13,   2379,    525,  11136,  16176,    389,  10951,
          29425,    315,   1467,     11,  10693,   1105,    311,   3960,  56116,
          12624,     11,  14389,     11,    323,  11871,   2878,   4128,     13,
           4220,   4119,    646,   2736,   5257,   5810,   4128,   8692,   9079,
             11,   1741,    438]])
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
A large language model is a type of artificial intelligence model designed 